In [2]:
import datetime
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

In [3]:
SCOPES = ['https://www.googleapis.com/auth/calendar']

In [4]:
def get_events_from_calendar():
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json')

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)

        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    try:
        service = build('calendar', 'v3', credentials=creds)
        now = datetime.datetime.now().isoformat() + 'Z'
        events_result = service.events().list(calendarId='primary', timeMin=now, maxResults=10, singleEvents=True, orderBy='startTime').execute()
        events = events_result.get('items', [])

        if not events:
            print('No upcoming events found.')

        for event in events:
            start = event['start'].get('dateTime', event['start'].get('date'))
            print(f'{start} {event["summary"]}')
    except HttpError as e:
        print(f'An error occurred: {e}')

In [6]:
get_events_from_calendar()

2025-03-07T14:30:00-05:00 Interview amazon sailesh anna
2025-03-07T17:00:00-05:00 Workout
2025-03-07T19:00:00-05:00 Business growth
2025-03-07T20:00:00-05:00 Libertty Prep
2025-03-07T20:00:00-05:00 FNH
2025-03-08T09:00:00-05:00 Apply jobs
2025-03-08T13:00:00-05:00 Mlops
2025-03-08T17:00:00-05:00 Workout
2025-03-08T19:00:00-05:00 Business growth
2025-03-08T20:00:00-05:00 Libertty Prep


In [7]:
def add_events_to_calendar(summary = 'Demo Event', location = 'Online', description = 'Demo Event', start_time = '2025-02-22T16:00:00', end_time = '2025-02-22T17:00:00', color_id = 1, attendees = [{"email": "sathwik238@gmail.com"}]):
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json')

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)

        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    try:
        service = build('calendar', 'v3', credentials=creds)
        
        event = {
            "summary": summary,
            "location": location,
            "description": description,
            "colorId" : color_id,
            "start" : {
                "dateTime": start_time,
                "timeZone": "EST"
            },
            "end" : {
                "dateTime": end_time,
                "timeZone": "EST"
            },
            "recurrence": [
                "RRULE:FREQ=DAILY;COUNT=1"
            ],
            "attendees": attendees
        }

        event = service.events().insert(calendarId='primary', body=event).execute()
        print(f'Event created: {event.get("htmlLink")}')


    except HttpError as e:
        print(f'An error occurred: {e}')

In [52]:
add_events_to_calendar(summary = 'Demo Event 1', location = 'Online', description = 'Demo Event', start_time = '2025-02-22T21:00:00', end_time = '2025-02-22T22:00:00', color_id = 1, attendees = [{"email": "sathwik238@gmail.com"}])


Event created: https://www.google.com/calendar/event?eid=djMwcWc3Y2E1amYzc25uOW5ia21rODR0ZWdfMjAyNTAyMjNUMDIwMDAwWiBzYXRod2lrMjM4QG0
